In [157]:
import pandas as pd
import numpy as np

-
- Year: Год выпуска или производства
- Model: Название или код модели автомобиля
- Car/Suv: Тип автомобиля (легковой автомобиль или внедорожник)
- Title: Название или описание автомобиля
- UsedOrNew: Состояние автомобиля (б/у или новый)
- Transmission: Тип трансмиссии (механическая или автоматическая)
- Engine: Объем двигателя или мощность (в литрах или киловаттах)
- DriveType: Тип привода (передний, задний или полный)
- FuelType: Тип топлива (бензин, дизель, гибрид или электричество)
- FuelConsumption: Потребление топлива (в литрах на 100 км)
- Kilometres: Пробег автомобиля (в километрах)
- ColourExtInt: Цвет автомобиля (внешний и внутренний)
- Location: Местонахождение автомобиля (город и штат)
- CylindersinEngine: Количество цилиндров в двигателе
- BodyType: Форма или стиль кузова автомобиля (седан, хэтчбек, купе и т.д.)
- Doors: Количество дверей в автомобиле
- Seats: Количество сидений в автомобиле

In [160]:
DATA_PATH ='../dataset/Australian_Vehicle_Prices.csv'

In [162]:
df = pd.read_csv(DATA_PATH)

In [164]:
df.head()

,Brand,Year,Model,Car/Suv,Title,UsedOrNew,Transmission,Engine,DriveType,FuelType,FuelConsumption,Kilometres,ColourExtInt,Location,CylindersinEngine,BodyType,Doors,Seats,Price
0,Ssangyong,2022.0,Rexton,Sutherland Isuzu Ute,2022 Ssangyong Rexton Ultimate (awd),DEMO,Automatic,"4 cyl, 2.2 L",AWD,Diesel,8.7 L / 100 km,5595,White / Black,"Caringbah, NSW",4 cyl,SUV,4 Doors,7 Seats,51990
1,MG,2022.0,MG3,Hatchback,2022 MG MG3 Auto Excite (with Navigation),USED,Automatic,"4 cyl, 1.5 L",Front,Premium,6.7 L / 100 km,16,Black / Black,"Brookvale, NSW",4 cyl,Hatchback,5 Doors,5 Seats,19990
2,BMW,2022.0,430I,Coupe,2022 BMW 430I M Sport,USED,Automatic,"4 cyl, 2 L",Rear,Premium,6.6 L / 100 km,8472,Grey / White,"Sylvania, NSW",4 cyl,Coupe,2 Doors,4 Seats,108988
3,Mercedes-Benz,2011.0,E500,Coupe,2011 Mercedes-Benz E500 Elegance,USED,Automatic,"8 cyl, 5.5 L",Rear,Premium,11 L / 100 km,136517,White / Brown,"Mount Druitt, NSW",8 cyl,Coupe,2 Doors,4 Seats,32990
4,Renault,2022.0,Arkana,SUV,2022 Renault Arkana Intens,USED,Automatic,"4 cyl, 1.3 L",Front,Unleaded,6 L / 100 km,1035,Grey / Black,"Castle Hill, NSW",4 cyl,SUV,4 Doors,5 Seats,34990


Посмотрю сначала на то, какой размерности датасет, есть ли пропущенные значения в нем

In [167]:
df.shape

(16734, 19)

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16734 entries, 0 to 16733
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Brand              16733 non-null  object 
 1   Year               16733 non-null  float64
 2   Model              16733 non-null  object 
 3   Car/Suv            16706 non-null  object 
 4   Title              16733 non-null  object 
 5   UsedOrNew          16733 non-null  object 
 6   Transmission       16733 non-null  object 
 7   Engine             16733 non-null  object 
 8   DriveType          16733 non-null  object 
 9   FuelType           16733 non-null  object 
 10  FuelConsumption    16733 non-null  object 
 11  Kilometres         16733 non-null  object 
 12  ColourExtInt       16733 non-null  object 
 13  Location           16284 non-null  object 
 14  CylindersinEngine  16733 non-null  object 
 15  BodyType           16452 non-null  object 
 16  Doors              151

Видно, что все признаки кроме одного числового - категориалные 

Рассмторим есть ли пропущенные значения в датасете 

In [173]:
pd.DataFrame(zip(df.isna().sum(),100 * (df.isna().sum()/df.shape[0])),columns=['Количество пропущенных значений','Процент от общего числа строк (%)'], index=df.columns)

,Количество пропущенных значений,Процент от общего числа строк (%)
Brand,1,0.005976
Year,1,0.005976
Model,1,0.005976
Car/Suv,28,0.167324
Title,1,0.005976
UsedOrNew,1,0.005976
Transmission,1,0.005976
Engine,1,0.005976
DriveType,1,0.005976
FuelType,1,0.005976


- Видно, что в каждый из признаков исходного датасета содержит пропущенные значения
- Можно выделить признаки Doors, Seats как те, которые содержат большое количество пропущенных значений (около 10 процентов от датасета)

Посмотрю теперь сумму пропущенных значений по строкам датасета 

In [177]:
missing_counts = df.isna().sum(axis=1)
missing_counts.sort_values(ascending=False)

2391     19
15802     4
15357     4
15351     4
15343     4
         ..
5905      0
5906      0
5907      0
5908      0
16733     0
Length: 16734, dtype: int64

- Вижу, что есть строка у которой пропущеннs все значения 
- Ее можно удалить , с остальными пропущенными значениями разбираться отдельно

In [180]:
df.iloc[2391]

Brand                NaN
Year                 NaN
Model                NaN
Car/Suv              NaN
Title                NaN
UsedOrNew            NaN
Transmission         NaN
Engine               NaN
DriveType            NaN
FuelType             NaN
FuelConsumption      NaN
Kilometres           NaN
ColourExtInt         NaN
Location             NaN
CylindersinEngine    NaN
BodyType             NaN
Doors                NaN
Seats                NaN
Price                NaN
Name: 2391, dtype: object

In [182]:
df = df.drop(2391)

Количество строк в датасете которые содержат хотя бы одно пропущенное значение

In [186]:
df[df.isna().sum(axis=1) > 0].shape[0]

2147